In [15]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [16]:
#Load dataset 
training_data = datasets.FashionMNIST(
    root ='data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root ='data',
    train = False,
    download=True,
    transform = ToTensor()
)

In [17]:
#Passo il Dataset come argomento al DataLoader

train_dataloader = DataLoader(training_data, batch_size=128)
test_dataloader = DataLoader(test_data, batch_size=128)

#Iterate through the DataLoader to access the images and labels of the samples
for X,y in test_dataloader:
    print('X shape :', {X.shape})
    print('y shape :',{y.shape})

    break

X shape : {torch.Size([128, 1, 28, 28])}
y shape : {torch.Size([128])}


In [18]:
#Creating the Model
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu= nn.Sequential(nn.Linear(28*28, 512), nn.ReLU(), nn.Linear(512,512), nn.ReLU(), nn.Linear(512,10))

    def forward(self, x):
        x = self.flatten(x)
        neural = self.linear_relu(x)

        return neural

model = NeuralNetwork().to(device)

print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [19]:
#Optimizing the Model Parameters
#Use CrossEntropy

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def training(dataloader, model, loss, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

def testing(dataloader , model, loss):
    size = len(dataloader.dataset)
    num_batcher = len(dataloader)
    #print(num_batcher)
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for X,y in dataloader:
            pred = model(X)
            test_loss += loss(pred,y).item()

            correct += (pred.argmax(1)== y).type(torch.float).sum().item()

    test_loss /= num_batcher
    correct /= size

    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [20]:
epochs = 10
for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    training(train_dataloader, model, loss_fn, optimizer)
    testing(test_dataloader, model, loss_fn)


Epoch 1
-------------------------------
loss: 2.293818 [    0/60000]
loss: 2.282549 [12800/60000]
loss: 2.270963 [25600/60000]
loss: 2.259310 [38400/60000]
loss: 2.235451 [51200/60000]
Test Error: 
 Accuracy: 30.3%, Avg loss: 2.225497 

Epoch 2
-------------------------------
loss: 2.223016 [    0/60000]
loss: 2.203103 [12800/60000]
loss: 2.192039 [25600/60000]
loss: 2.182838 [38400/60000]
loss: 2.148236 [51200/60000]
Test Error: 
 Accuracy: 39.3%, Avg loss: 2.136339 

Epoch 3
-------------------------------
loss: 2.133516 [    0/60000]
loss: 2.099137 [12800/60000]
loss: 2.086856 [25600/60000]
loss: 2.074003 [38400/60000]
loss: 2.030259 [51200/60000]
Test Error: 
 Accuracy: 46.3%, Avg loss: 2.011365 

Epoch 4
-------------------------------
loss: 2.005684 [    0/60000]
loss: 1.952584 [12800/60000]
loss: 1.938406 [25600/60000]
loss: 1.916204 [38400/60000]
loss: 1.872706 [51200/60000]
Test Error: 
 Accuracy: 51.7%, Avg loss: 1.843296 

Epoch 5
-------------------------------
loss: 1.8308

In [26]:
#Now we can save make predictions
values = [
    "T-shirt/top",
    "Ankle boot",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandals",
    "Shirt",
    "Sneakers",
    "Bag",
    
]

model.eval()
#prendo coppie di valori
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted = values[pred[0].argmax(0)]
    actual = values[y]

    print("Predicted :", predicted, "\tActual value:", actual)

Predicted : Bag 	Actual value: Bag
